In [1]:
import pandas as pd
import numpy as np
import pulp
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Load the dataset
data = pd.read_csv("retail_sales_dataset.csv")

In [3]:
# Feature engineering
data['Total Amount'] = data['Quantity'] * data['Price per Unit']

In [4]:
# Splitting dataset into features and target variable
X = data[['Age', 'Quantity', 'Price per Unit']]  # Independent variables
y = data['Total Amount']  # Dependent variable (revenue)

In [5]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Train a regression model
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [7]:
# Evaluate the model
y_pred = model.predict(X_test)
print("Model Performance:")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R2 Score: {r2_score(y_test, y_pred):.2f}")

Model Performance:
Mean Squared Error: 0.00
R2 Score: 1.00


In [8]:
# Predict revenue for the entire dataset
data['Predicted Revenue'] = model.predict(X)

In [9]:
# Optimization setup
transactions = data['Transaction ID']
model_optimization = pulp.LpProblem("Revenue_Optimization", pulp.LpMaximize)

In [10]:
# Decision variables: quantity to allocate to each transaction
decision_vars = pulp.LpVariable.dicts("Quantity", transactions, lowBound=0, cat='Continuous')

In [11]:
# Objective function: Maximize total predicted revenue
revenue_dict = data.set_index('Transaction ID')['Predicted Revenue'].to_dict()
model_optimization += pulp.lpSum([revenue_dict[txn] * decision_vars[txn] for txn in transactions]), "Total Predicted Revenue"

In [12]:
# Constraints: Quantity constraints (quantity allocated cannot exceed available quantity)
quantity_dict = data.set_index('Transaction ID')['Quantity'].to_dict()
for txn in transactions:
    model_optimization += decision_vars[txn] <= quantity_dict[txn], f"Quantity_Constraint_{txn}"


In [13]:
# Solve the optimization problem
model_optimization.solve()

1

In [14]:
# Output the results
if model_optimization.status == pulp.LpStatusOptimal:
    print("Optimal Total Predicted Revenue:", pulp.value(model_optimization.objective))
    
    # Storing results for better readability
    results = []
    for txn in transactions:
        allocated_qty = pulp.value(decision_vars[txn])
        if allocated_qty > 0:
            results.append({
                "Transaction ID": txn,
                "Allocated Quantity": allocated_qty,
                "Predicted Revenue": allocated_qty * revenue_dict[txn]
            })
            print(f"Transaction ID: {txn}, Allocated Quantity: {allocated_qty:.0f}, "
                  f"Predicted Revenue: {allocated_qty * revenue_dict[txn]:.2f}")
    
    # Convert results to a DataFrame for better visualization
    results_df = pd.DataFrame(results)
    print("\nOptimal Allocation Results:\n", results_df)
    
else:
    print("No optimal solution found.")

Optimal Total Predicted Revenue: 1383200.0
Transaction ID: 1, Allocated Quantity: 3, Predicted Revenue: 450.00
Transaction ID: 2, Allocated Quantity: 2, Predicted Revenue: 2000.00
Transaction ID: 3, Allocated Quantity: 1, Predicted Revenue: 30.00
Transaction ID: 4, Allocated Quantity: 1, Predicted Revenue: 500.00
Transaction ID: 5, Allocated Quantity: 2, Predicted Revenue: 200.00
Transaction ID: 6, Allocated Quantity: 1, Predicted Revenue: 30.00
Transaction ID: 7, Allocated Quantity: 2, Predicted Revenue: 100.00
Transaction ID: 8, Allocated Quantity: 4, Predicted Revenue: 400.00
Transaction ID: 9, Allocated Quantity: 2, Predicted Revenue: 1200.00
Transaction ID: 10, Allocated Quantity: 4, Predicted Revenue: 800.00
Transaction ID: 11, Allocated Quantity: 2, Predicted Revenue: 200.00
Transaction ID: 12, Allocated Quantity: 3, Predicted Revenue: 225.00
Transaction ID: 13, Allocated Quantity: 3, Predicted Revenue: 4500.00
Transaction ID: 14, Allocated Quantity: 4, Predicted Revenue: 480.00